- Possible raesons for latency in pdr-llama2-chromadb code
  - llama2 downloaded in local computer
  - slow parent document retriever
  - small cpu
- Giskard for evaluation, opensource
  - By default uses gpt4 but llama3 also could be connected
  - it finds contexts from pdf/url and generates question, further context is matched to the RAG response for evaluating RAG.
- llama3 could be taken from groq or ollama
- llama3(robust) vs mixtral-8x7b-32768(speciafic task)
- FAISS works fast with GPUs, could be replaced by chromaDB
- For now, this is not conversational like previous one

In [ ]:
a = 100
print(a)

# Installing Dependencies

In [1]:
!pip install -q langchain
!pip install -q langchain-core
!pip install -q langchain-community
!pip install -q fastembed
!pip install -q pypdf
!pip install -q langchain_groq
!pip install -q faiss-gpu
!pip install -q fastembed

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpys

### Importing Libraries

In [2]:
import time
import pandas as pd
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.base import Chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from IPython.display import Markdown, display

### Environment Variables

In [3]:
pd.set_option("display.max_colwidth", None)

path1 = "/kaggle/input/pdf-files/GB AEBS.pdf"
path2 = "/kaggle/input/pdf-files/UN AEBS.pdf"

mixtral = ChatGroq(groq_api_key ="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",
               model = 'mixtral-8x7b-32768',
              temperature=0.1)

llama3 = ChatGroq(groq_api_key ="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",
               model = 'llama3-70b-8192',
              temperature=0.1)

llms = [llama3, mixtral]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

# For RAG1
qa_template = """
Please answer the question solely based on the provided context.
If the exact answer is not found in the context but related information is available, respond with "I can't find the exact answer, but these may be helpful:" and then provide the relevant information.
Accuracy and precision are crucial for this task.
Your response is very crutial for my life, response wisely.

CONTEXT: {context}

QUESTION: {question}
"""

# For RAG1
comparison_template = """
    We have provided a question and their two answers.
    Generate a comparison section without a heading which includes whether both answer are same or partially same or different. If they are paritially same, then what is same and what is different. This comparison is based on the answers generated from both the contexts.
    Accuracy and precision are crucial for this task.

    QUESTION: {question}

    ANSWER 1: {answer1}

    ANSWER 2: {answer2}
    """

qa_prompt = PromptTemplate(template=qa_template, input_variables=["question", "context"])

# For RAG2
def get_comparision_prompt(query, context1, context2):
    comparison_template = """
    Response in three sections
    
    ANSWER 1: This is firts section, here answer the question form the context 1.
    
    ANSWER 2: This is second section, here answer the question form the context 2.
    
    COMPARISON: This is third section, here answer whether both answer are same or partially same or different. If they are paritially same, then what is same and what is different.
    This section is completely based on answer generated in first and second section.
    
    Please answer the question solely based on the provided context.
    If you can't answer any of the both questions from their context then just tell that there is no answer in that context.
    This is very important for my life, be very precised and accurate in answering the question and also in comparison..

    QUESTION: {question}

    CONTEXT1: {context1}

    CONTEXT2: {context2}
    """
    comparison_prompt = comparison_template.format(context1 = context1,context2 = context2, question = query)
    return comparison_prompt

# Get vectorstore as retriever

In [4]:
def get_retriever(path):
    docs = PyPDFLoader(path).load_and_split(text_splitter)
    vectorstore = FAISS.from_documents(docs, FastEmbedEmbeddings())
    retriever = vectorstore.as_retriever()
    return retriever

In [5]:
retriever1 = get_retriever(path1)
retriever2 = get_retriever(path2)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# RAG 1
- using RetrievalQA
- by invoking LLM 3 times

In [18]:
def RAG1(query):
    llm = mixtral
    
    qa_chain1 = RetrievalQA.from_llm(llm=llm, retriever=retriever1, prompt= qa_prompt)
    qa_chain2 = RetrievalQA.from_llm(llm=llm, retriever=retriever2, prompt= qa_prompt)

    answer1 = qa_chain1.invoke(query)['result']
    answer2 = qa_chain2.invoke(query)['result']
    
    comparison_prompt = comparison_template.format(question = query,answer1 = answer1, answer2 = answer2)
    comparison = llm.invoke(comparison_prompt).content
    
    response = f"**ANSWER 1**: {answer1}\n\n**ANSWER 2**: {answer2}\n\n**COMPARISION**: {comparison}"
    return response
    

In [14]:
retriever2.invoke("Explain the test procedures in detail.")

[Document(page_content='exper tise from all the system fields which are involved. \nFor periodic technical inspections, the documentation shall descr ibe how the current operational status of ‘The \nSyste m’ can be check ed. \n3.1.1.  Documentation shall be made available in two parts: \n(a)  The formal documentation packag e for the appro val, containing the mater ial listed in paragraph 3 of this \nannex (with the excep tion of that of paragraph 3.4.4 belo w) whic h shall be supplied to the Techn ical \nService at the time of submission of the type appro val application. This will be taken as the basic reference \nfor the verification process set out in paragraph 4 of this annex. \n(b) Additional mate rial and analysis data of paragraph 3.4.4 belo w, which shall be retained by the manufa c\xad\nturer , but made open for inspection at the time of type appro val. \n3.2.  Descr iption of the functions of ‘The System ’', metadata={'source': '/kaggle/input/pdf-files/UN AEBS.pdf', 'page': 

In [7]:
query = "What AEBS should do in vehicle ignition?"
s = time.time()
display(Markdown(RAG1(query)))
print(time.time()-s)

**ANSWER 1**: According to the context, when the AEBS of a vehicle is deactivated, it shall automatically resume the normal work status upon vehicle ignition (4.5.4 a).

**ANSWER 2**: Based on the provided context, I can find the exact answer:

According to paragraph 5.4.1, "The AEBS function shall be automatically reinstated at the initiation of each new ignition cycle."

So, when the vehicle ignition is turned on, the AEBS function should be automatically reinstated.

**COMPARISION**: Both answers convey the same message, which is that the AEBS function should be automatically reinstated or resume its normal work status when the vehicle ignition is turned on. The key similarity between the two answers is that they both indicate that the AEBS function is reactivated upon vehicle ignition. The difference lies in the specific wording and the referenced paragraph numbers, with Answer 1 mentioning "normal work status" and referencing paragraph 4.5.4 a, while Answer 2 uses the phrase "automatically reinstated" and references paragraph 5.4.1.

4.137349605560303


In [21]:
s = time.time()
display(Markdown(RAG1("Explain the test procedures in detail, in detail.")))
print(time.time()-s)

**ANSWER 1**: I'm sorry for any confusion, but the provided context does not include detailed test procedures. However, I can provide some general steps based on the information given:

1. Perform function tests under non-fault state of the vehicle system, as per the function concept in A.2.2.1, to determine the normal operation level of AEBS (Advanced Emergency Braking System).

2. For function safety concept confirmation, simulate the influence of internal fault of an assembly by applying relevant output signals to the electronic/electrical or mechanical assembly of AEBS. Inspect the reaction of AEBS in case of failure of a single assembly.

3. Compare the results with the conclusion of the function safety concept and describe the sufficiency of relevant safety concepts and their implementation effects.

4. If necessary, perform function tests as per special test procedures specified in this standard or other standards.

5. For validation test support, describe how to inspect the work state under normal operation and failure mode of AEBS.

Please note that these are general steps and the exact procedures may vary depending on the specific requirements and the system being tested.

**ANSWER 2**: I can't find the exact answer in the context, but these may be helpful:

The test procedures for the system, as outlined in the context, include:

1. Verification of the function of the system: This involves testing the normal operational levels of the system under non-fault conditions. The performance of the system is verified against the manufacturer's basic benchmark specification, unless there is a specified performance test as part of the approval procedure of this or another Regulation.

2. Verification of the safety concept: The reaction of the system is checked under the influence of a failure in any individual unit. Corresponding output signals are applied to electrical units or mechanical elements to simulate the effects of internal faults within the unit. The verification results should correspond with the documented summary of the failure analysis, to a level of overall effect that confirms the adequacy of the safety concept and execution.

3. Verification of the performance of the vehicle system: This is conducted against the manufacturer's basic benchmark specification.

4. Verification of the reaction of the system under the influence of a failure in any individual unit: This is done to check the safety concept of the system.

5. Inspection of the methods and tools used in the design and development process: The manufacturer should be able to show evidence of the means by which they determined the realization of the system logic during the design and development process.

6. Explanation of the design provisions built into the system: The manufacturer should provide an explanation of the design provisions built into the system to generate safe operation under fault conditions. Examples of design provisions for failure in the system include fall-back to operation using a partial system, change-over to a separate backup system, and removal of the high-level function. In case of a failure, the driver should be warned by a warning signal or message display.

**COMPARISION**: The two answers provided discuss the test procedures for a system, specifically the Advanced Emergency Braking System (AEBS), in detail. Both answers highlight the importance of performing function tests under non-fault state of the vehicle system and verifying the function of the system under normal operational levels. They also mention the need to verify the safety concept by simulating the influence of internal faults and inspecting the reaction of the system in case of failure of a single assembly or unit.

However, there are some differences between the two answers. Answer 1 provides more specific steps for the test procedures, such as describing the sufficiency of relevant safety concepts and their implementation effects, performing function tests as per special test procedures specified in this standard or other standards, and inspecting the work state under normal operation and failure mode of AEBS. Answer 2, on the other hand, includes additional steps such as verification of the performance of the vehicle system against the manufacturer's basic benchmark specification, verification of the reaction of the system under the influence of a failure in any individual unit, inspection of the methods and tools used in the design and development process, and explanation of the design provisions built into the system.

In summary, while both answers discuss the test procedures for a system in detail, Answer 1 provides more specific steps for the test procedures, while Answer 2 includes additional steps related to the verification of the performance of the vehicle system, inspection of the methods and tools used in the design and development process, and explanation of the design provisions built into the system.

12.587482929229736


In [22]:
query = "What are warning indications, in details."
s = time.time()
display(Markdown(RAG1(query)))
print(time.time()-s)

**ANSWER 1**: I can't find the exact answer in the provided context, but these may be helpful:

According to the context, there are several warning indications mentioned. Here are the details:

1. Optical warning signal for failure: The constant ON yellow warning signal specified in GB 4094 -2016 shall be used for warning signal for failure (4.2.4, 4.2.5, 4.2.7, and 4.4).
2. Collision early warning: The collision early warning shall be provided by at least two kinds of signals selected from acoustic, haptic or optical signals (4.2.2).
3. Flashing warning signal for failure: The flashing warning signal specified in 4.2.4 may be adopted for collision early warning (4.2.3).
4. Optical early warning: The optical early warning and warning signals shall be visible even in daytime (4.2.6).
5. Deactivation warning: It is necessary to send out a deactivation warning when AEBS manual is deactivated, and the warning signal shall be in compliance with the stipulations of 4.2.4 (November 2019 ACEA Translation).

These warning indications are used to alert the driver of different system statuses or potential risks.

**ANSWER 2**: I can't find the exact answer in the provided context, but these may be helpful:

The warning indications are part of the Advanced Emergency Braking System (AEBS) safety requirements in the regulation. According to the context, there are at least three types of warning indications related to the AEBS:

1. Collision warning: This warning is provided when the AEBS has detected the possibility of a collision with a preceding vehicle in the same lane which is traveling at a slower speed, has slowed to a halt, or is stationary having not been identified as moving. The warning shall be provided by at least two modes selected from acoustic, haptic, or optical, and the timing of the warning signals shall be such that they provide the possibility for the driver to react to the risk of collision.
2. Failure warning: This warning is provided when there is a failure in the AEBS that prevents the requirements of this Regulation from being met. In case of an electrically detectable failure, there shall not be an appreciable time interval between each AEBS self-check, and subsequently, there shall not be an appreciable delay in illuminating the warning signal.
3. Yellow optical warning signal: This signal is provided when the AEBS is temporarily not available, for example, due to inclement weather conditions. The signal shall be constant and yellow in color.

Moreover, the optical warning signals shall be visible even by daylight, and the satisfactory condition of the signals must be easily verifiable by the driver from the driver's seat.

**COMPARISION**: Both answers provide a detailed explanation of the warning indications related to the Advanced Emergency Braking System (AEBS) and include several similar warning indications. They are both unable to find an exact answer in the provided context but provide helpful information.

The first answer provides five warning indications, including the optical warning signal for failure, collision early warning, flashing warning signal for failure, optical early warning, and deactivation warning. The second answer provides three warning indications, including collision warning, failure warning, and yellow optical warning signal.

The collision warning and failure warning in the second answer are similar to the collision early warning and flashing warning signal for failure in the first answer. However, the first answer provides more details on the types of signals used for collision early warning and the timing of the warning signals. The second answer provides more details on the failure warning, including the requirement for an electrically detectable failure and the absence of an appreciable time interval between each AEBS self-check.

The yellow optical warning signal in the second answer is similar to the optical warning signal for failure in the first answer. However, the first answer provides more details on the color and visibility of the signal.

Overall, both answers are partially similar, with some differences in the types of warning indications provided and the details included.

4.033109903335571


In [17]:
query = "What are warning indications, in details."
s = time.time()
display(Markdown(RAG2(query)))
print(time.time()-s)

**ANSWER 1:**
In the context of Context 1, warning indications refer to optical warning signals that are used to alert the driver of potential failures or issues with the AEBS system. These warnings are specified in section 4.2.4 and include:

* A constant ON yellow warning signal specified in GB 4094-2016 for warning signals for failure.
* A flashing warning signal for failure that may be adopted.
* An optical warning signal that is visible even in daylight and is easily verifiable by the driver from the driver's seat.

**ANSWER 2:**
In the context of Context 2, warning indications refer to warning signals that are used to alert the driver of potential collisions or system failures. These warnings are specified in section 5.5 and include:

* A collision warning that is provided by at least two modes selected from acoustic, haptic, or optical signals.
* A failure warning that is specified in paragraph 5.5.4, which may be a yellow warning signal.
* An optical warning signal that is visible even in daylight and is easily verifiable by the driver from the driver's seat.

**COMPARISON:**
Both Context 1 and Context 2 describe warning indications as optical warning signals used to alert the driver of potential issues or failures with the AEBS system. However, there are some differences between the two contexts.

* Both contexts specify that the warning signals should be visible even in daylight and easily verifiable by the driver from the driver's seat.
* Both contexts mention the use of yellow warning signals, but Context 1 specifies a constant ON yellow warning signal, while Context 2 does not specify the type of yellow warning signal.
* Context 1 mentions the use of flashing warning signals, which is not mentioned in Context 2.
* Context 2 provides more details on the types of warnings, including collision warnings and failure warnings, whereas Context 1 focuses more on the specifications of the warning signals themselves.

Overall, while both contexts describe warning indications as optical warning signals, there are some differences in the specifics of the warnings and their specifications.

2.6997921466827393


# RAG 2
- Retrieval ----> context from scratch`
- will help in custom prompting for comparison

In [8]:
def get_context(query, retriever):
    retrieved_docs = retriever.invoke(query)
    context = ""
    for doc in retrieved_docs:
        context += doc.page_content
    return context

In [9]:
def RAG2(query):
    llm = llama3
    
    context1 = get_context(query, retriever1)
    context2 = get_context(query, retriever2)
    
    prompt = get_comparision_prompt(query, context1, context2)
    
    return llm.invoke(prompt).content

In [13]:
s = time.time()
display(Markdown(RAG2("Explain the test procedures in detail.")))
print(time.time()-s)

**ANSWER 1:**

Based on Context 1, the test procedures involve:

* Function test under non-fault state of vehicle system to determine normal operation level of AEBS.
* Simulating the influence of internal fault of assembly through application of relevant output signal to electronic/electric assembly or mechanical assembly of AEBS, to inspect the reaction of AEBS in case of failure of single assembly.
* Confirming that results shall be identical with the conclusion of function safety concept and describing the sufficiency of relevant safety concepts and their implementation effects.
* Inspecting the work state under normal operation and failure mode of AEBS.

**ANSWER 2:**

Based on Context 2, the test procedures involve:

* Verification of the functions of 'The System' under non-fault conditions against the manufacturer's basic benchmark specification.
* Verification of the safety concept by simulating the effects of internal faults within the unit by applying corresponding output signals to electrical units or mechanical elements.
* Testing the functional operation of 'The System' as laid out in the documents required in paragraph 3 above.
* Conducting tests under specific conditions, such as flat, dry concrete or asphalt surface, ambient temperature between 0 °C and 45 °C, horizontal visibility range, and no wind.

**COMPARISON:**

Both answers partially overlap, but they have some differences.

Similarities:

* Both contexts mention testing the system under non-fault conditions.
* Both contexts involve simulating internal faults to inspect the reaction of the system.

Differences:

* Context 1 focuses on AEBS (Automatic Emergency Braking System) and its specific requirements, while Context 2 is more general and refers to 'The System' without specifying what it is.
* Context 1 emphasizes the importance of function safety concept and its implementation effects, while Context 2 does not mention this aspect.
* Context 2 provides more detailed information about the test conditions, such as the surface, temperature, and visibility range, which are not mentioned in Context 1.

In summary, while both contexts share some similarities, they have distinct differences in their focus, scope, and specific requirements.

3.030167579650879


In [10]:
query1 = "When collision early warning signal shall be sent?"
s = time.time()
display(Markdown(RAG2(query1)))
print(time.time() - s)

**ANSWER 1:**
According to Context 1, a collision early warning signal shall be sent when the AEBS has detected the possibility of a collision with a forward vehicle of category M, N, or O in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary (4.2.1). The timing of the early warning signal shall be in compliance with the stipulations of 4.2.2, which states that the warning signal shall enable the driver to react to the risk of collision and take control of the vehicle timely, while avoiding nuisance for the driver by too early or too frequent early warnings.

**ANSWER 2:**
According to Context 2, a collision warning shall be provided when the AEBS has detected the possibility of a collision with a preceding vehicle of category M, N, or O in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary (5.2.1.1). The timing of the warning signals shall be such that they provide the possibility for the driver to react to the risk of collision and take control of the situation, and shall also avoid nuisance for the driver by too early or too frequent warnings (5.5.1).

**COMPARISON:**
Both answers are partially same. In both contexts, a collision early warning signal shall be sent when the AEBS detects the possibility of a collision with a forward or preceding vehicle in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary. The timing of the warning signals is also similar, aiming to enable the driver to react to the risk of collision and take control of the vehicle timely, while avoiding nuisance for the driver by too early or too frequent early warnings. However, the specific stipulations and requirements for the warning signals differ slightly between the two contexts.

2.4995875358581543


In [11]:
query2 = "When collision early warning signal shall be sent?"
s = time.time()
display(Markdown(RAG2(query2)))
print(time.time() - s)

**ANSWER 1:**
According to Context 1, a collision early warning signal shall be sent when the AEBS has detected the possibility of a collision with a forward vehicle of category M, N, or O in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary (4.2.1). The timing of the early warning signal shall be in compliance with the stipulations of 4.2.2, which states that the warning signal shall enable the driver to react to the risk of collision and take control of the vehicle timely, while avoiding nuisance for the driver by too early or too frequent early warnings.

**ANSWER 2:**
According to Context 2, a collision warning shall be provided when the AEBS has detected the possibility of a collision with a preceding vehicle of category M, N, or O in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary (5.2.1.1). The timing of the warning signals shall be such that they provide the possibility for the driver to react to the risk of collision and take control of the situation, and shall also avoid nuisance for the driver by too early or too frequent warnings (5.5.1).

**COMPARISON:**
Both answers are partially same. In both contexts, a collision early warning signal shall be sent when the AEBS detects the possibility of a collision with a forward or preceding vehicle in the same lane, which is traveling at a slower speed, has slowed to a halt, or is stationary. The timing of the warning signals is also similar, aiming to enable the driver to react to the risk of collision and take control of the vehicle timely, while avoiding nuisance for the driver by too early or too frequent early warnings. However, the specific stipulations and requirements for the warning signals differ slightly between the two contexts.

8.455300569534302


### Question-answering with PDF 1
GB AEBS

In [ ]:
queries1 = ["What is hapting?" ,"why you can't tell me about machine learning form you knowledge database?","When collision early warning signal shall be sent?", "What AEBS should do in vehicle ignition?","The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than how many kilometers per hour?"]

In [ ]:
qa_chain1 = RetrievalQA.from_llm(llm=llama3, retriever=retriever1, prompt= qa_prompt)
for query in queries1:
    st = time.time()
    print("QUERY:", query)
    print("RESPONSE:", qa_chain1.invoke(query)['result'])
    print("++++ generated in", round((time.time() - st),2), "sec")
    print("")